In [1]:
import matplotlib.image as mpimg
import numpy as np
import cv2
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from skimage.feature import hog
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  block_norm= 'L2-Hys',
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       block_norm= 'L2-Hys',
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features



# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features
    
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.7, 0.7)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy


import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import os
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
#from lesson_functions import *
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
# from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split

# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    if color_space != 'RGB':
        if color_space == 'HSV':
            img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            img = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            img = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: img = np.copy(img) 

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

    
# Read in cars and notcars
images_car = glob.glob(os.path.join('C:\\Users\\HX\\Self_drive_car_Project_5\\data\\car\\**\\','*.png'))
images_nocar = glob.glob(os.path.join('C:\\Users\\HX\\Self_drive_car_Project_5\\data\\nocar\\**\\','*.png'))
#print(images)
cars = []
notcars = []
for image in images_car:
    cars.append(image)
    
for image in images_nocar:
    notcars.append(image)

        
# Reduce the sample size because
# The quiz evaluator times out after 13s of CPU time
"""
sample_size = 3
cars = cars[0:sample_size]

notcars = notcars[0:sample_size]
"""
### TODO: Tweak these parameters and see how the results change.
color_space = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (12, 12) # Spatial binning dimensions
hist_bins = 48    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [400, 656] # Min and max in y to search in slide_window()

car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

#a3 = np.shape(car_features)
#print (a3)
#print (b3)


#a3,b3 = np.shape(notcar_features)
#print (a3)
#print (b3)


# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float32)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rand_state)
    
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()

svc.fit(X_train, y_train)
svc.fit
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()












from sklearn.externals import joblib #jbolib模块

#保存Model(注:save文件夹要预先建立，否则会报错)
#joblib.dump(svc, 'svc.pkl')

#读取Model
#svc = joblib.load('svc.pkl')


#image = mpimg.imread('bbox-example-image.jpg')
image = cv2.imread('test_image.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
draw_image = np.copy(image)
#print(image)
#image = mpimg.imread('test_image.jpg')
print(image)
#image = image.astype(np.float32)/255
#print(image)


# Uncomment the following line if you extracted training
# data from .png images (scaled 0 to 1 by mpimg) and the
# image you are searching is a .jpg (scaled 0 to 255)
#image = image.astype(np.float32)/255


"""
windows_1 = slide_window(image, x_start_stop=[640, None], y_start_stop=[400, 520], 
                    xy_window=(64, 64), xy_overlap=(0.7, 0.7))

windows_2 = slide_window(image, x_start_stop=[640, None], y_start_stop=[450, 580], 
                    xy_window=(96, 96), xy_overlap=(0.7, 0.7))

windows_3 = slide_window(image, x_start_stop=[640, None], y_start_stop=[500, 660], 
                    xy_window=(128, 128), xy_overlap=(0.7, 0.7))

windows_4 = slide_window(image, x_start_stop=[640, None], y_start_stop=[550, 720], 
                    xy_window=(160, 160), xy_overlap=(0.7, 0.7))
"""

windows_1 = slide_window(image, x_start_stop=[720, None], y_start_stop=[400, 477], 
                    xy_window=(64, 64), xy_overlap=(0.85, 0.85))

windows_2 = slide_window(image, x_start_stop=[720, None], y_start_stop=[400, 115], 
                    xy_window=(96, 96), xy_overlap=(0.8, 0.8))

windows_3 = slide_window(image, x_start_stop=[720, None], y_start_stop=[400, 554], 
                    xy_window=(128, 128), xy_overlap=(0.7, 0.7))

windows_4 = slide_window(image, x_start_stop=[720, None], y_start_stop=[400, 592], 
                    xy_window=(160, 160), xy_overlap=(0.6, 0.6))

windows=windows_1 + windows_2 + windows_3 + windows_4
#print(windows)
#print(windows)
hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)                       
print(hot_windows)
window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    

plt.imshow(window_img)
plt.show()






























import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
from scipy.ndimage.measurements import label

# Read in a pickle file with bboxes saved
# Each item in the "all_bboxes" list will contain a 
# list of boxes for one of the images shown above
#box_list = pickle.load( open( "bbox_pickle.p", "rb" ))
box_list = hot_windows
print(box_list)
# Read in image similar to one shown above 
#image = mpimg.imread('bbox-example-image.jpg')
image = cv2.imread('test_image.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
heat = np.zeros_like(image[:,:,0]).astype(np.float)

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

# Add heat to each box in box list
heat = add_heat(heat,box_list)
    
# Apply threshold to help remove false positives
heat = apply_threshold(heat,1)

# Visualize the heatmap when displaying    
heatmap = np.clip(heat, 0, 255)

# Find final boxes from heatmap using label function
labels = label(heatmap)
draw_img = draw_labeled_bboxes(np.copy(image), labels)

fig = plt.figure()
plt.subplot(211)
plt.imshow(draw_img)
plt.title('Car Positions')
plt.subplot(212)
plt.imshow(heatmap, cmap='hot')
plt.title('Heat Map')
fig.tight_layout()
plt.show()






from moviepy.editor import VideoFileClip
from IPython.display import HTML
existing_folder_name=''
New_foler_name=''

video_name='test_video'
video_name='project_video'

#video_name='solidYellowLeft'
#video_name='challenge'

clip = VideoFileClip(video_name+'.mp4') 
white_output = video_name+'_result.mp4'

def process_image(image):
    draw_image = np.copy(image)
    #draw_image = cv2.imread(image)
    image=draw_image
    #print(image)
    #image_1 = mpimg.imread(image)
    #image = draw_image.astype(np.float32)/255
    #print(image)
    
    # Uncomment the following line if you extracted training
    # data from .png images (scaled 0 to 1 by mpimg) and the
    # image you are searching is a .jpg (scaled 0 to 255)
    #image = image.astype(np.float32)/255


    """
    windows_1 = slide_window(image, x_start_stop=[640, None], y_start_stop=[400, 520], 
                        xy_window=(64, 64), xy_overlap=(0.7, 0.7))

    windows_2 = slide_window(image, x_start_stop=[640, None], y_start_stop=[450, 580], 
                        xy_window=(96, 96), xy_overlap=(0.7, 0.7))

    windows_3 = slide_window(image, x_start_stop=[640, None], y_start_stop=[500, 660], 
                        xy_window=(128, 128), xy_overlap=(0.7, 0.7))

    windows_4 = slide_window(image, x_start_stop=[640, None], y_start_stop=[550, 720], 
                        xy_window=(160, 160), xy_overlap=(0.7, 0.7))
    """

    windows_1 = slide_window(image, x_start_stop=[720, None], y_start_stop=[400, 477], 
                        xy_window=(64, 64), xy_overlap=(0.85, 0.85))

    windows_2 = slide_window(image, x_start_stop=[720, None], y_start_stop=[400, 115], 
                        xy_window=(96, 96), xy_overlap=(0.8, 0.8))

    windows_3 = slide_window(image, x_start_stop=[720, None], y_start_stop=[400, 554], 
                        xy_window=(128, 128), xy_overlap=(0.7, 0.7))

    windows_4 = slide_window(image, x_start_stop=[720, None], y_start_stop=[400, 592], 
                        xy_window=(160, 160), xy_overlap=(0.6, 0.6))

    windows=windows_1 + windows_2 + windows_3 + windows_4
    #print(windows)
    hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)                       
    #print(hot_windows)
    window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    


    #image = window_img
    
    box_list = hot_windows
    #print(box_list)
    # Read in image similar to one shown above 
    #image = mpimg.imread('bbox-example-image.jpg')
    #image = mpimg.imread('test_image.png')

    heat = np.zeros_like(draw_image[:,:,0]).astype(np.float)
    
    # Add heat to each box in box list
    heat = add_heat(heat,box_list)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,1)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    fianl_img = draw_labeled_bboxes(np.copy(draw_image), labels)
    image=fianl_img
    return image

#video
white_clip = clip.fl_image(process_image) #NOTE: this function expects color images!!
white_clip.write_videofile(white_output, audio=False)


D:\Software\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 5868
16.32 Seconds to train SVC...
Test Accuracy of SVC =  0.9747
[[[ 48  57  64]
  [ 36  46  55]
  [ 33  45  57]
  ...
  [181 182 186]
  [215 211 208]
  [248 241 235]]

 [[ 29  47  61]
  [ 21  40  55]
  [ 62  83 100]
  ...
  [137 153 166]
  [155 164 173]
  [192 197 203]]

 [[ 11  41  65]
  [ 74 106 129]
  [125 159 186]
  ...
  [124 155 183]
  [132 158 181]
  [136 158 179]]

 ...

 [[ 75  65  64]
  [ 99  87  87]
  [100  88  88]
  ...
  [ 94  77  70]
  [106  89  82]
  [142 125 118]]

 [[ 65  55  54]
  [104  92  92]
  [107  95  95]
  ...
  [ 82  64  60]
  [101  83  79]
  [133 118 113]]

 [[ 34  24  23]
  [ 78  68  67]
  [112 100 100]
  ...
  [ 75  57  53]
  [116 101  96]
  [149 134 129]]]
[((819, 400), (883, 464)), ((828, 400), (892, 464)), ((837, 400), (901, 464)), ((1017, 400), (1081, 464)), ((1062, 400), (1126, 464)), ((1080, 400), (1144, 464)), ((1089, 400), (1153, 464)), ((1098, 400), (1162, 464)), 

<Figure size 640x480 with 1 Axes>

[((819, 400), (883, 464)), ((828, 400), (892, 464)), ((837, 400), (901, 464)), ((1017, 400), (1081, 464)), ((1062, 400), (1126, 464)), ((1080, 400), (1144, 464)), ((1089, 400), (1153, 464)), ((1098, 400), (1162, 464)), ((1116, 400), (1180, 464)), ((1125, 400), (1189, 464)), ((1134, 400), (1198, 464)), ((1143, 400), (1207, 464)), ((1152, 400), (1216, 464)), ((1161, 400), (1225, 464)), ((1170, 400), (1234, 464)), ((1179, 400), (1243, 464)), ((1188, 400), (1252, 464)), ((1197, 400), (1261, 464)), ((1215, 400), (1279, 464)), ((810, 409), (874, 473)), ((981, 409), (1045, 473)), ((990, 409), (1054, 473)), ((999, 409), (1063, 473)), ((1008, 409), (1072, 473)), ((1044, 409), (1108, 473)), ((1053, 409), (1117, 473)), ((1062, 409), (1126, 473)), ((1071, 409), (1135, 473)), ((1089, 409), (1153, 473)), ((1098, 409), (1162, 473)), ((1107, 409), (1171, 473)), ((1125, 409), (1189, 473)), ((1143, 409), (1207, 473)), ((1152, 409), (1216, 473)), ((1161, 409), (1225, 473)), ((1170, 409), (1234, 473)), ((

<Figure size 640x480 with 2 Axes>

[MoviePy] >>>> Building video project_video_result.mp4
[MoviePy] Writing video project_video_result.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [16:13<00:00,  1.29it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_result.mp4 

